### 1. Importing evertything 

In [23]:

#importing part

import torch
import torch.nn as nn

import numpy as np
import os
import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import torchvision.transforms.functional as fn   
import torch
from scipy.stats import entropy

from torchvision.utils import save_image 



from torch.utils.tensorboard import SummaryWriter
import torch.nn as n


### 2. Setting dataset folder and Constants

In [24]:
# CONSTANTS
BATCH_SIZE = 128
IMG_CHANNELS=3
NUM_EPOCHS=2
Z_DIM=100
LR = 2e-4
IMAGE_SIZE = 64
FEATURES_GEN =64
FEATURES_DISC=64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
folder_path = '/home/metaphysicist/Coding/Research/Font_Generation/data/Images/Images_dir/a/'
#folder_path = '/home/metaphysicist/Coding/Research/Font_Generation/Georgian_dataset/data/'
image_shape = (32,32,3) # 
nm_imgs = np.sort(os.listdir(folder_path)) # sorting files in folder


##### 2.1 some options

In [25]:
# Experiments: print specific letter folder
print(nm_imgs)

['A']


In [26]:
normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

Transforms = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.5 for _ in range(IMG_CHANNELS)], [0.5 for _ in range(IMG_CHANNELS)]
        ),
    ]
)

Dataset = datasets.ImageFolder(root=folder_path,transform=Transforms)

Dataset_Loader = DataLoader(Dataset,batch_size=BATCH_SIZE,shuffle=True) 

### 3. Transforming, Getting And Visualising each photo and their parameters  

* For running data images plotting process uncomment plot_img function call

In [27]:
def plot_img(loader):
    for batch_idx, (real, i) in enumerate(loader):


        
        # print(batch_idx, (real, i))
        # imgTensor, labels = next(loader_iter)

        img_grid_real = torchvision.utils.make_grid(
                            real[:1], normalize=True
                        )

        img_grid_real = img_grid_real.permute(1, 2, 0)
        # Set up plot config
        plt.figure(figsize=(8, 2), dpi=300)
        plt.axis('off')

        # Plot Image Grid
        plt.imshow(img_grid_real)


        # Show the plot
        plt.show()

# plot_img(Dataset_Loader)
##### For running data images plotting process uncomment plot_img function call [upper]

### 4. Building Models And Setting Weight Generation

In [28]:
class Discriminator(n.Module):
    def __init__(self,channels_img,features_dim):
        super(Discriminator,self).__init__()
        self.disc = n.Sequential(
            # Input is N x channels_img x 64 x 64
            n.Conv2d(in_channels=channels_img, out_channels=features_dim, kernel_size=4,stride=2,padding=1),
            # 32 x 32 
            n.LeakyReLU(0.2),
            
            self._block(features_dim, features_dim*2, 4, 2, 1),
            # 16 x 16
            self._block(features_dim*2, features_dim*4, 4, 2, 1),
            # 8 x 8
            self._block(features_dim*4, features_dim*8, 4, 2, 1),
            # # 4 x 4
            n.Conv2d(features_dim*8, 1, kernel_size=4, stride=2, padding=0),
            # 1 x 1
            n.Sigmoid(),
        )
    
    def _block(self,input_features,output_features,kernel,stride,padding):
        return n.Sequential(
            n.Conv2d(input_features,output_features,kernel,stride,padding,bias=False),
            n.BatchNorm2d(output_features),
            n.LeakyReLU(0.2),
        )
    
    def forward(self,x):
        return self.disc(x)


class Generator(n.Module):
    def __init__(self,z_dim,channels_img,features_g):
        super(Generator,self).__init__()
        self.gen = n.Sequential(
            # Input N x z_dim x 1 x 1
            self._block(z_dim, features_g*16, kernel=4, stride=1, padding=0),
            # N x f_g*16 x 4 x 4
            self._block(features_g*16, features_g*8, kernel=4, stride=2, padding=1),
            # 8x8
            self._block(features_g*8,features_g*4,kernel=4,stride=2,padding=1),
            # 4x4
            self._block(features_g*4, features_g*2, kernel=4, stride=2, padding=1),
            # # 2x2
            n.ConvTranspose2d(features_g*2, channels_img, kernel_size=4, stride=2, padding=1),
            n.Tanh()
        )
    
    def _block(self,input_channels,output_channels,kernel,stride,padding):
        return n.Sequential(
            n.ConvTranspose2d(input_channels, output_channels,kernel, stride, padding),
            n.BatchNorm2d(output_channels),
            n.ReLU()
        )
    
    def forward(self,x):
        return self.gen(x)

def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, (n.Conv2d,n.BatchNorm2d,n.ConvTranspose2d)):
            n.init.normal_(m.weight.data,0.0,0.02)

### 5. Training and Implementing PART I

In [29]:
gen = Generator(Z_DIM,IMG_CHANNELS,FEATURES_GEN).to(device)
disc = Discriminator(IMG_CHANNELS,FEATURES_DISC).to(device)
initialize_weights(gen)
initialize_weights(disc)
opt_gen = torch.optim.Adam(gen.parameters(),lr=LR,betas=(0.5,0.999))
opt_disc = torch.optim.Adam(disc.parameters(),lr=LR,betas=(0.5,0.999))
criterion = nn.BCELoss()

fixed_noise = torch.randn(BATCH_SIZE,Z_DIM,1,1).to(device)
writer_real = SummaryWriter("logs/real")
writer_fake = SummaryWriter("logs/fake")
gen.train()
disc.train()

Discriminator(
  (disc): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (4): Sequential(
      (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (5): Conv2d(512, 1, kernel_size=(4, 4), stride=(2, 2))
    (6): Sigmoid()
  )
)

### INCEPTION SCORE START

In [30]:


# import torch
# from torch import nn
# import numpy as np
# from scipy.stats import entropy

# def inception_score(real_images, fake_images, inception_model, batch_size=64, num_splits=10):
#     """
#     Calculates the Inception Score (IS) to evaluate the quality of generated images.

#     Args:
#         real_images (torch.Tensor): Real images as a tensor.
#         fake_images (torch.Tensor): Fake/generated images as a tensor.
#         inception_model (torch.nn.Module): Pre-trained Inception model.
#         batch_size (int): Batch size for feeding images to the Inception model.
#         num_splits (int): Number of splits to calculate the IS. More splits yield a more accurate score.

#     Returns:
#         float: Inception Score.
#     """

#     # Set device (GPU if available, else CPU)
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     print(device)
#     # Set the model to evaluation mode
#     inception_model.eval()

#     # Calculate the number of samples
#     num_samples = real_images.size(0)

#     # Split the images into batches
#     real_batches = torch.split(real_images, batch_size)
#     fake_batches = torch.split(fake_images, batch_size)

#     # Initialize the list to store the scores for each split
#     split_scores = []

#     # Calculate the Inception Score for each split
#     for i in range(num_splits):
#         # Get a batch of real and fake images
#         real_batch = real_batches[i].to(device)
#         fake_batch = fake_batches[i].to(device)

#         # Compute the logits for real and fake images
        
#         with torch.no_grad():
#             real_logits = inception_model(real_batch).to(device)
#             fake_logits = inception_model(fake_batch).to(device)

#         # Compute the softmax probabilities for real and fake images
#         real_probs = torch.softmax(real_logits, dim=1)
#         fake_probs = torch.softmax(fake_logits, dim=1)

#         # Compute the marginal entropy of the softmax probabilities
#         real_entropy = torch.sum(-real_probs * torch.log(real_probs + 1e-8), dim=1)
#         fake_entropy = torch.sum(-fake_probs * torch.log(fake_probs + 1e-8), dim=1)

#         # Compute the average entropy for each batch
#         avg_real_entropy = torch.mean(real_entropy)
#         avg_fake_entropy = torch.mean(fake_entropy)

#         # Calculate the Inception Score for the current split
#         split_score = torch.exp(avg_real_entropy - avg_fake_entropy)
#         split_scores.append(split_score.item())

#     # Calculate the final Inception Score as the exponential of the mean of the split scores
#     is_mean = np.mean(split_scores)
#     is_std = np.std(split_scores)
#     inception_score = is_mean

#     return inception_score

In [31]:
import torch
import torchvision
from torchvision import transforms
from scipy.linalg import sqrtm

def calculate_inception_score_and_fid(generated_images, real_images):

    inception_model = torchvision.models.inception_v3(pretrained=True, transform_input=False, aux_logits=True)
    inception_model = inception_model.eval()

    # Preprocess the generated images
    preprocess = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard Inception normalization
    ])
    generated_images = torch.stack([preprocess(image) for image in generated_images])
    
    # Get the predictions for the generated images
    predictions = inception_model(generated_images)

    # Calculate the IS
    probabilities = torch.nn.functional.softmax(predictions, dim=1)
    entropy = torch.sum(probabilities * torch.log(probabilities), dim=1)
    inception_score = torch.exp(torch.mean(entropy))

    # Preprocess the real images
    real_images = torch.stack([preprocess(image) for image in real_images])

    # Extract the feature representations from the Inception model
    features_real = inception_model(real_images).detach().numpy()
    features_generated = predictions.detach().numpy()

    # Calculate the mean and covariance for real and generated features
    mean_real = np.mean(features_real, axis=0)
    mean_generated = np.mean(features_generated, axis=0)
    cov_real = np.cov(features_real, rowvar=False)
    cov_generated = np.cov(features_generated, rowvar=False)

    # Calculate the FID using the Fréchet distance
    diff = mean_real - mean_generated
    sqrt_cov_product = sqrtm(cov_real.dot(cov_generated))
    fid = np.real(diff.dot(diff) + np.trace(cov_real + cov_generated - 2 * sqrt_cov_product))

    return fid.item()

In [32]:
# inception_model = torchvision.models.inception_v3(pretrained=True, transform_input=False, aux_logits=True)


### INCEPTION SCORE FINISH

In [33]:
def process(epochs=NUM_EPOCHS):
    step=0
    for epoch in range(epochs):
        for batch_idx, (real, i) in enumerate(Dataset_Loader):
            real = real.to(device)
            noise = torch.randn(BATCH_SIZE,Z_DIM,1,1).to(device)
            fake = gen(noise).to(device)
            ### Train Discriminator max log(D(x)) + log(1 - D(G(z)))
            disc_real = disc(real).reshape(-1) # giving proper shape
            loss_disc_real =  criterion(disc_real, torch.ones_like(disc_real)) # counting specificly real loss
            disc_fake = disc(fake.detach()).reshape(-1) # giving shape
            
            loss_disc_fake = criterion(disc_fake,torch.zeros_like(disc_fake)) # counting specificly fake loss
            loss_disc = (loss_disc_real+loss_disc_fake) / 2 #overall loss
            disc.zero_grad() # gradient descent
            loss_disc.backward(retain_graph=True)
            opt_disc.step()

            ### Train Generator min  log(1- D(G(z))) <---> max log(D(G(z)))
            output = disc(fake)
            loss_gen = criterion(output,torch.ones_like(output))
            gen.zero_grad()
            loss_gen.backward()
            opt_gen.step()

            

            if batch_idx %100==0:
                print(
                    f"Epoch [{epoch}/{epochs}] Batch {batch_idx}/{len(Dataset_Loader)} \
                            Loss D: {loss_disc:.4f}, loss G: {loss_gen:.4f}"
                )
                with torch.no_grad():
                    fake = gen(fixed_noise).to(device)

                    val = calculate_inception_score_and_fid(fake,real)
                    print(val)

                    print("Freschet Inception Distance less than 1000")

                    img_grid_real = torchvision.utils.make_grid(
                        real[:32], normalize=True
                    )
                    img_grid_fake = torchvision.utils.make_grid(
                        fake[:32], normalize=True
                    )

                    writer_fake.add_image("Fake", img_grid_fake,global_step=step)
                    writer_real.add_image("Real", img_grid_real,global_step=step)
                    
                    if val<1200:
                        
                        ###### Image Saving
                        FAKE_MNIST_FOLD = "E" + str(epoch) + "_S" + str(step) + "_"
                        FOLDER_PATH = f"/home/metaphysicist/Coding/Research/Font_Generation/Latest_Results/{FAKE_MNIST_FOLD}"
                        os.mkdir(FOLDER_PATH)
                        for i in range(len(fake)):
                            save_image(fake[i], f"{FOLDER_PATH}/fake{i}.png")
                        print("fake",len(fake))
                        
                        ###### Image Saving
                        
                    # print(len(fake))
                step += 1
process(20)
# torch.save(gen.state_dict(), 'generator1.pth')
# torch.save(disc.state_dict(), 'discriminator1.pth')

Epoch [0/20] Batch 0/118                             Loss D: 0.7004, loss G: 0.7887
1861.990803568836
Freschet Inception Distance less than 1000
Epoch [0/20] Batch 100/118                             Loss D: 0.0144, loss G: 4.1529
1503.157092744716
Freschet Inception Distance less than 1000
Epoch [1/20] Batch 0/118                             Loss D: 0.0108, loss G: 4.4544
1521.7171256830648
Freschet Inception Distance less than 1000
Epoch [1/20] Batch 100/118                             Loss D: 0.7567, loss G: 1.9168
1497.6841438778058
Freschet Inception Distance less than 1000
Epoch [2/20] Batch 0/118                             Loss D: 0.5103, loss G: 1.1008
1313.7678065705466
Freschet Inception Distance less than 1000
Epoch [2/20] Batch 100/118                             Loss D: 0.5871, loss G: 1.5588
1264.6165769039003
Freschet Inception Distance less than 1000
Epoch [3/20] Batch 0/118                             Loss D: 0.5131, loss G: 1.2438
1075.358969621932
Freschet Inception

In [ ]:

# g = Generator(Z_DIM,IMG_CHANNELS,FEATURES_GEN).to(device)
# d = Discriminator(IMG_CHANNELS,FEATURES_GEN).to(device)
# g.load_state_dict(torch.load('generator.pth'))
# d.load_state_dict(torch.load('discriminator.pth'))

# # Generate a bunch of fake images
# num_samples = 100
# latent_samples = torch.randn(BATCH_SIZE,Z_DIM,1,1).to(device)
# generated_images = g(latent_samples)

# inception_model = torchvision.models.inception_v3(pretrained=True, transform_input=False, aux_logits=True)
# inception_model.eval()

# # Calculate Inception Scores for each generated image
# inception_scores = []

# for image in generated_images:
    
#     image_resized = torch.nn.functional.interpolate(torch.from_numpy(image.cpu()).unsqueeze(0),
#                                                     size=(299, 299), mode='bilinear', align_corners=False)
#     image_preprocessed = (image_resized - 0.5) / 0.5

#     with torch.no_grad():
#         logits = inception_model(image_preprocessed)

#     softmax_probs = torch.softmax(logits, dim=1).numpy()
#     kl_divergence = entropy(np.mean(softmax_probs, axis=0), base=2)
#     inception_score = np.exp(kl_divergence)

#     inception_scores.append(inception_score)

# # Print Inception Scores for each generated image
# for i, score in enumerate(inception_scores):
#     print("Inception Score for image", i + 1, ":", score)